In [1]:
import ROOT
from analysis_framework import Dataset, Analysis

In [2]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad
n_threads = 12
# prod = False
prod = True
no_rvec = True
dataset_path = "data/datasets/miniDSTs/processed-test.json"
output_path = "root://eosproject.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/test"
output_meta = "test.json"
checked_output_meta = "checked-test.json"
output_collections = r"^(?!R2jet_lvecs)\w+$" # ["PandoraPFOs"]
plot_dir = "plots/pre-selection/test"
if prod:
    dataset_path = "data/datasets/miniDSTs/processed-no-exc-higgs.json"
    output_path = "root://eosproject.cern.ch//eos/experiment/clicdp/data/user/l/lreichen/snapshots3/full"
    output_meta = "full.json"
    checked_output_meta = "checked-full.json"
    output_collections = r"^(?!R2jet_lvecs)\w+$"
    plot_dir = "plots/pre-selection/full"


In [3]:
# ROOT.gInterpreter.GenerateDictionary("ROOT::RVec<ROOT::Math::PxPyPzEVector>", "Math/Vector4D.h;ROOT/RVec.hxx")

In [4]:
ROOT.EnableImplicitMT(n_threads)

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = Analysis(dataset)

analysis.init_parameters([
    ("WWCategorisation.RecoCatBasic", "int", "8"),
    ("WWCategorisation.RecoCatAdvanced", "int", "8"),
    ("WWCategorisation.missE", "float", "-42"),
    ("WWCategorisation.misspT", "float", "-42"),
    ("WWCategorisation.mInv", "float", "-42"),
    ("Energy", "float", "-42"),
])

TypeError: none of the 10 overloaded methods succeeded. Full details:
  RDataFrame::RDataFrame(string_view treename, initializer_list<string> filenames, const ROOT::RDataFrame::ColumnNames_t& defaultColumns = {}) =>
    invalid_argument: RDataFrame: could not open file "/eos/experiment/clicdp/data/user/l/lreichen/miniDST/2f/2f_eehiq/l5_o1/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I500003.P2f_z_eehiq.eR.pR.n063.d_dstm_15918_0_mini-DST.edm4hep.root".
  RDataFrame::RDataFrame(ROOT::RDataFrame&&) =>
    TypeError: takes at most 1 arguments (2 given)
  RDataFrame::RDataFrame(string_view treeName, TDirectory* dirPtr, const ROOT::RDataFrame::ColumnNames_t& defaultColumns = {}) =>
    TypeError: could not convert argument 2
  RDataFrame::RDataFrame(TTree& tree, const ROOT::RDataFrame::ColumnNames_t& defaultColumns = {}) =>
    TypeError: could not convert argument 1
  RDataFrame::RDataFrame(const ROOT::RDataFrame&) =>
    TypeError: takes at most 1 arguments (2 given)
  RDataFrame::RDataFrame(s

Error in <TNetXNGFile::Open>: [ERROR] Server responded with an error: [3014] Unable to open file  /eos/experiment/clicdp/data/user/l/lreichen/miniDST/2f/2f_eehiq/l5_o1/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I500003.P2f_z_eehiq.eR.pR.n063.d_dstm_15918_0_mini-DST.edm4hep.root; Network is unreachable

SysError in <TFile::ReadBuffer>: error reading from file /eos/experiment/clicdp/data/user/l/lreichen/miniDST/2f/2f_eehiq/l5_o1/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I500003.P2f_z_eehiq.eR.pR.n063.d_dstm_15918_0_mini-DST.edm4hep.root Remote I/O error
Error in <TFile::Init>: /eos/experiment/clicdp/data/user/l/lreichen/miniDST/2f/2f_eehiq/l5_o1/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I500003.P2f_z_eehiq.eR.pR.n063.d_dstm_15918_0_mini-DST.edm4hep.root failed to read the file type data.
Error in <TNetXNGFile::Open>: [ERROR] Server responded with an error: [3014] Unable to open file  /eos/experiment/clicdp/data/user/l/lreichen/miniDST/2f/2f_eehiq/l5_o1/rv02-02.sv02-02.mILD_l5_o1_v02.E250-Set

In [ ]:
# signal cut goes here:
# In this case, we want to require the charged lepton in the final state to be inside of the detector acceptance
# just take the first gen status 1 particle with a fitting pdg value and cut on theta
lep_pdg = 11
# CLD 150 mrad
# ILD ~84 mrad (IDR: 4.8 deg)
acceptance_theta = 0.084
signal_cut = f"""
std::invoke([](auto& genStat, auto& pdg, auto& px, auto& py, auto& pz, auto& m) -> bool {{
auto lepton_mask = genStat == 1 && abs(pdg) == {lep_pdg};
// abuse ArgMax to get the first set position
auto lepton_idx = ArgMax(lepton_mask);
auto lepton_lvec = ROOT::Math::PxPyPzMVector(px[lepton_idx], py[lepton_idx],
                                             pz[lepton_idx], m[lepton_idx]);
double lepton_theta = lepton_lvec.Theta();
return abs(cos(lepton_theta)) < cos({acceptance_theta});
}}, MCParticlesSkimmed.generatorStatus, MCParticlesSkimmed.PDG, MCParticlesSkimmed.momentum.x, MCParticlesSkimmed.momentum.y, MCParticlesSkimmed.momentum.z, MCParticlesSkimmed.mass)
"""

In [ ]:
analysis.set_categories({
    "4f_sw_sl_signal": {"pattern": "4f_sw_sl", "cut": signal_cut},
    "4f_sl_bkg": {"pattern": r"4f\w+sl", "cut": None }, # inverse signal cut will be applied automatically
    "4f_not_sl": {"pattern": r"4f\w+_(?:h|l)", "cut": None },
    # separate out aa_4f? super low lumi anyway
    "aa2f": {"pattern": "aa_2f", "cut": None},
    # 2f but not aa_2f
    "2f": {"pattern": "(?<!aa_)2f", "cut": None},
    "3f": {"pattern": "ea_3f|ae_3f", "cut": None},
    "5f": {"pattern": "ea_5f|ae_5f", "cut": None},
    "6f": {"pattern": "6f", "cut": None},
    # need to filter out anything ending in _h and 2f_z_eehiq
    "higgs": {"pattern": "[^_e]h", "cut": None},
})
# check if we missed any processes
print(analysis.is_complete_categorisation())

In [ ]:
print(list(analysis._df.keys()))

In [ ]:
# needed for the .size() calls... alternative would probably be to .Alias the @size columns
ROOT.gInterpreter.Declare("#include <podio/ObjectID.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/ReconstructedParticleData.h>")
analysis.Define("n_isomuons", "IsolatedMuons_objIdx.size()")
analysis.Define("n_isoelectrons", "IsolatedElectrons_objIdx.size()")
analysis.Define("RecoCatBasic", "params_WWCategorisation_RecoCatBasic")
analysis.Define("RecoCatAdvanced", "params_WWCategorisation_RecoCatAdvanced")
analysis.Define("n_charged_PFOs", "Sum(abs(PandoraPFOs.charge) == 1.)")
analysis.Define("n_R2jets", "Refined2Jets.size()")
make_lvec = lambda coll_name: f"ROOT::VecOps::Construct<ROOT::Math::PxPyPzEVector>({coll_name}.momentum.x, {coll_name}.momentum.y, {coll_name}.momentum.z, {coll_name}.energy)"
analysis.Define("R2jet_lvecs", make_lvec("Refined2Jets"))
analysis.Define("M_jj", "ROOT::VecOps::Sum(R2jet_lvecs, ROOT::Math::PxPyPzEVector()).mass()")
analysis.Define("PVertex_ndf", "PrimaryVertex.ndf")
analysis.Define("PVertex_chi2", "PrimaryVertex.chi2")
analysis.Define("PVertex_chi2ndf", "PVertex_chi2 / PVertex_ndf")

signal_category = ["4f_sw_sl_signal"]
# analysis.define_truth_objects(signal_category)
# analysis.define_reco_objects(x_angle)
# analysis.remove_x_angle(x_angle)
# analysis.Define("M_Wlep", "ub_leptonic_W_lvec.M()")
# analysis.Define("M2_Wlep_sign", "M_Wlep / abs(M_Wlep)")

In [ ]:
analysis.book_histogram_1D("RecoCatBasic", "RecoCatBasic", ("", ";RecoCatBasic", 8, 0., 8.))
# analysis.book_histogram_1D("RecoCatAdvanced", "RecoCatAdvanced", ("", ";RecoCatAdvanced", 8, 0., 8.))
# analysis.add_filter("RecoCatAdvanced == 2", "RecoCatAdvanced == 2")

In [ ]:
analysis.book_histogram_1D("n_isoelectrons", "n_isoelectrons", ("", ";n_isoelectrons", 5, 0., 5.))
analysis.add_filter("n_isoelectrons == 1", "1 iso electron")

analysis.book_histogram_1D("n_charged_PFOs", "n_charged_PFOs", ("", ";n_charged_PFOs", 50, 0., 50.))
analysis.add_filter("n_charged_PFOs >= 10", "N_{#pm}#geq 10")

analysis.book_histogram_1D("misspT", "params_WWCategorisation_misspT", ("", ";misspT", 100, 0., 100.))
analysis.add_filter("params_WWCategorisation_misspT >= 10", "misspT#geq 10")

analysis.book_histogram_1D("RecoCatAdvanced", "RecoCatAdvanced", ("", ";RecoCatAdvanced", 8, 0., 8.))
analysis.add_filter("RecoCatAdvanced == 2", "RecoCat = 2")

# analysis.book_histogram_1D("M2_Wlep_sign", "M2_Wlep_sign", ("", ";M2_Wlep_sign", 5, -1.5, 3.5))
# analysis.add_filter("M2_Wlep_sign > 0", "M2_Wlep_sign > 0")

analysis.book_histogram_1D("PVertex_ndf", "PVertex_ndf", ("", ";PVertex_ndf", 50, 1., 101.))
analysis.book_histogram_1D("missE", "params_WWCategorisation_missE", ("", ";missE", 300, -50., 250.))
analysis.book_histogram_1D("mInv", "params_WWCategorisation_mInv", ("", ";mInv", 100, 40., 140.))
analysis.book_histogram_1D("M_jj", "M_jj", ("", ";M_jj", 100, 32., 132.))
analysis.book_histogram_1D("n_isomuons", "n_isomuons", ("", ";n_isomuons", 5, 0., 5.))
analysis.book_histogram_1D("PVertex_chi2", "PVertex_chi2", ("", ";PVertex_chi2", 75, 0., 150.))
analysis.book_histogram_1D("PVertex_chi2ndf", "PVertex_chi2ndf", ("", ";PVertex_chi2/ndf", 20, 0., 10.))

In [ ]:
analysis.book_reports()

In [ ]:
# analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [ ]:
%%time
analysis.run()

In [ ]:
# analysis.check_snapshots("events", output_path, checked_output_meta)

In [ ]:
analysis._calc_cutflow()

In [ ]:
analysis.print_reports()
analysis.draw_cutflow(plot_dir=plot_dir)

In [ ]:
# unfortunately the jsroot plots break all the time, maybe it is just too much for them
# %jsroot on
# 0 - hadronic (both W decay hadronically)
# 1 - invisible semileptonic (one W decays hadronically, but the lepton is not within detector acceptance/detected)
# 2 - semileptonic electron
# 3 - semileptonic muon
# 4 - semileptonic tauon
# 5 - leptonic (both W decay leptonically)
# 6 - other, 7- broken?
# analysis.draw_histogram("RecoCatBasic")
analysis.draw_histogram("n_isoelectrons", logY=True, plot_dir=plot_dir, x_arrowl=1.0, x_arrowr=2.0)
analysis.draw_histogram("n_charged_PFOs", logY=True, plot_dir=plot_dir, x_arrowl=10.)
analysis.draw_histogram("misspT", logY=True, plot_dir=plot_dir, x_arrowl=10.)
analysis.draw_histogram("RecoCatAdvanced", plot_dir=plot_dir, x_arrowl=2., x_arrowr=3.)
# analysis.draw_histogram("M2_Wlep_sign", plot_dir=plot_dir, x_arrowl=0.5)

In [ ]:
analysis.draw_histogram("PVertex_ndf")

analysis.draw_histogram("missE")
analysis.draw_histogram("mInv")
analysis.draw_histogram("M_jj")
analysis.draw_histogram("n_isomuons")

In [ ]:
analysis.draw_histogram("PVertex_chi2")
analysis.draw_histogram("PVertex_chi2ndf")